In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
import codecs
import time
import re
import os
import json
import time

.

In [2]:
def getdatafromsite(line, linetype):  # Scrapping data from site
    inputline = line
    inputtype = linetype
    problem = []
    pbwords = []
    s_type = {}
    s_type['WX'] = 'WX'
    s_type['SLP'] = 'SL'
    s_type['Velthuis'] = 'VH'
    s_type['KH'] = 'KH'

    s_d = inputline

    s_c = s_d.replace(" ", "+")
    # for utilising the sanskrit heritage app, the url has been specified
    urlname = ("http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=f&us=f&cp=t&text=" +
               s_c + "&t=" + s_type[inputtype] + "&topic=&mode=g&corpmode=&corpdir=&sentno=")

    print(urlname)
    #time.sleep(2)
    page = requests.get(urlname)
    # parsing using beautifulsoup
    soup = bs(page.text, 'html.parser')
    table = soup.table
    tablebody = table.find('table', {'class': 'center'})
    t = pd.DataFrame(
        columns=['id', 'level', 'color_class', 'position', 'chunk_no', 'word', 'lemma', 'pre_verb', 'morph', 'colspan',
                 'wordlenth', 'aux_inf'])

    i = 0
    id_ = 0
    if not (tablebody):  #### wronginputs
        print('no table body of given inputline')

    # for valid entries corresponding to Wordsinsentence
    for child in tablebody.children:
        if (child.name == 'tr'):
            if i < 1:
                linechar = []
                c = 0
                for char in child.children:
                    linechar.append(char.string)
                    c += 1
                i += 1
                line_header = "".join(linechar)
                linechunks = line_header.split("\xa0")
                continue
            position_ = 0
            j = 0
            for wordtable in child.children:
                c = 0
                for ch in linechar[0:position_]:
                    if (re.match('\xa0', ch)):  # or (re.match('_',ch))
                        c += 1
                    # if the contents exist in wordtable
                    # following assignings are carried out.
                if (wordtable.contents):
                    color_ = wordtable.table.get('class')[0]
                    colspan_ = wordtable.get('colspan')
                    word_ = wordtable.table.tr.td.string
                    onclickdatas_ = wordtable.table.tr.td.get('onclick')
                    for onclickdata_ in onclickdatas_.split("<br>"):  # required splits carried out at positions stated
                        morphslist_ = re.findall(r'{ (.*?) }', onclickdata_)  # .split(' | ')
                        ldata = str(re.search(r'{.*?}\[(.*)\]', onclickdata_).group(1))
                        ldata = str(re.sub(r'</?a.*?>|</?i>', "", ldata))

                        lemmadata = ldata.split(" ")
                        if len(lemmadata) > 1:
                            auxi_ = " ".join(lemmadata[1:])
                        else:
                            auxi_ = ""
                        lemmas_ = "".join(lemmadata[0])
                        lemmalists_ = lemmas_.split("-")
                        if (len(lemmalists_) > 1):
                            preverb_ = ",".join(lemmalists_[0:(len(lemmalists_) - 1)])
                            lemmalist_ = "".join(lemmalists_[-1:]).split("_")
                        else:
                            preverb_ = ""
                            lemmalist_ = "".join(lemmalists_[0]).split("_")
                        if (len(lemmalist_) > 1):
                            auxi_ = auxi_ + " sence of lemma = " + "".join(lemmalist_[1:(len(lemmalist_))])
                            lemma_ = "".join(lemmalist_[0])
                        else:
                            lemma_ = "".join(lemmalist_[0])
                        morphs_ = str(morphslist_[0])
                        for morph_ in morphs_.split(" | "):
                            t.loc[id_] = [id_, i, color_, position_, c + 1, word_, lemma_, preverb_, morph_,
                                          int(colspan_), len(word_), auxi_]
                            if (re.match(r'grey_back', color_)):
                                if not (word_ == 'pop'):
                                    problem.append(id_)  # filling entries to problem list
                                else:
                                    id_ = id_ - 1
                            id_ += 1

                    position_ += int(colspan_)
                else:
                    position_ += 1
            i = i + 1
            dict_ = {'t':t,'line_header':line_header}
    return dict_

In [3]:
#!/usr/bin/env python3

# Double characters mapping to single characters
_dbl = dict({
    'ai' : 'E',
    'au' : 'O',
    'kh' : 'K',
    'gh' : 'G',
    'ch' : 'C',
    'jh' : 'J',
    'ṭh' : 'W',
    'ḍh' : 'Q',
    'th' : 'T',
    'dh' : 'D',
    'ph' : 'P',
    'bh' : 'B'})

# One to one mapping
_oth = dict({
    'ā' : 'A',
    'ī' : 'I',
    'ū' : 'U',
    'ṛ' : 'f',
    'ṝ' : 'F',
    'ḷ' : 'x',
    'ḹ' : 'X',
    'ṃ' : 'M',
    'ḥ' : 'H',
    'ṁ' : '~',
    'ṅ' : 'N',
    'ñ' : 'Y',
    'ṭ' : 'w',
    'ḍ' : 'q',
    'ṇ' : 'R',
    'ś' : 'S',
    'ṣ' : 'z'})

def iast2slp(src):
    '''
    Converts International Alphabet for Sanskrit Transliteration (IAST) scheme to
    Sanskrit Library Phonetic Basic notation
    '''
    tgt = ''
    inc = 0
    while inc < len(src):
        now = src[inc]
        nxt = src[inc+1] if inc < len(src) - 1 else ''
        if now + nxt in _dbl:
            tgt += _dbl[now + nxt]
            inc += 1
        elif now in _oth:
            tgt += _oth[now]
        else:
            tgt += now
        inc += 1
    return tgt

    

In [4]:
data_pack = getdatafromsite('AcaraRIyam', 'SLP')

http://sanskrit.inria.fr/cgi-bin/SKT/sktgraph?lex=SH&st=f&us=f&cp=t&text=AcaraRIyam&t=SL&topic=&mode=g&corpmode=&corpdir=&sentno=


In [5]:
data_pack['t']

,id,level,color_class,position,chunk_no,word,lemma,pre_verb,morph,colspan,wordlenth,aux_inf
0,0,1,deep_sky_back,0,1,ācaraṇīyam,caraṇīya,ā,acc. sg. n.,10,10,{ pfp. [2] }[ā-car]
1,1,1,deep_sky_back,0,1,ācaraṇīyam,caraṇīya,ā,nom. sg. n.,10,10,{ pfp. [2] }[ā-car]
2,2,1,deep_sky_back,0,1,ācaraṇīyam,caraṇīya,ā,acc. sg. m.,10,10,{ pfp. [2] }[ā-car]


In [6]:
data_pack['t']['lemma']

0    caraṇīya
1    caraṇīya
2    caraṇīya
Name: lemma, dtype: object